In [2]:
# Importamos las librerías que necesitamos

# Librerías de extracción de datos
# -----------------------------------------------------------------------

%load_ext autoreload
%autoreload 2

# Importaciones:
# Beautifulsoup
from bs4 import BeautifulSoup

# Requests
import requests

import pandas as pd
import numpy as np

from time import sleep

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 

import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_columns', None)

## Tabla principal de Suicidios

In [4]:
driver = webdriver.Chrome()
url_suicidios = "https://www.one.gob.do/datos-y-estadisticas/"
driver.get(url_suicidios)
driver.maximize_window()

sleep(1)

try:
    driver.find_element("xpath", '/html/body/div[7]/section/div/ul/li[2]/a').click()
except:
    print('No encuentro el botón')

sleep(2)

for i in range(3):
    try:
        driver.execute_script("window.scrollBy(0, 400);")  # Desplazarse 1000 píxeles hacia abajo
        sleep(2)
    except:
        print('No he podido descender')

try: 
    driver.find_element("xpath", '/html/body/div[7]/section/div/div[3]/div[2]/div/div[2]/div[2]/div/div/div[2]/header/div[2]').click()
except:
    print('No encuentro el botón para ampliar muertes')
sleep(2)

try: 
    driver.find_element("xpath", '/html/body/div[7]/section/div/div[3]/div[2]/div/div[2]/div[2]/div/div/div[2]/div/header/div[1]').click()
except:
    print('No encuentro el botón para abrir pestañas')
sleep(2)

try: 
    driver.find_element("xpath", '/html/body/div[7]/section/div/div[3]/div[2]/div/div[2]/div[2]/div/div/div[2]/div/div/div/div/div[2]/div/table/tbody/tr[5]/td/a/div[2]/h5').click()
except:
    print('No encuentro el botón para descargar el diccionario de suicidios')
sleep(2)

try:
    driver.execute_script("window.scrollBy(0, 400);")  # Desplazarse 1000 píxeles hacia abajo
    sleep(2)
except:
    print('No he podido descender')

try: 
    driver.find_element("xpath", '/html/body/div[7]/section/div/div[3]/div[2]/div/div[2]/div[2]/div/div/div[2]/div/div/div/div/div[2]/div/table/tbody/tr[15]/td/a/div[2]/h5').click()
except:
    print('No encuentro el botón para descargar el diccionario de suicidios')
sleep(2)

## Tablas secundarias

In [42]:
driver = webdriver.Chrome()
url_defunciones = "file:///C:/Users/DELL/Downloads/Libro%20de%20c%C3%B3digos%20de%20variable%20de%20defunciones%20por%20suicidios%202007_2023.htm"
driver.get(url_defunciones)
driver.maximize_window()

sleep(1)  # Esperar un segundo entre cada scroll

In [43]:
sopa_defunciones = BeautifulSoup(driver.page_source) # saco la sopa de tablas
lista_tablas = sopa_defunciones.findAll('div', {'class': 'itemBody'})

dataframes = []

for i, tabla in enumerate(lista_tablas):
    rows = tabla.find_all('tr')

    data = []
    for row in rows:
        cells = row.find_all('td') #saco el contenido de cada celda
        data.append([cell.get_text(strip=True) for cell in cells if cell.get_text(strip=True)])

    df = pd.DataFrame(data)

    dataframes.append(df) #paso el df a una lista de dfs

print(f"Se generaron {len(dataframes)} DataFrames.")

Se generaron 12 DataFrames.


In [44]:
## saco los dfs de manera individual
df_region = dataframes[1] 
df_provincia = dataframes[2]
df_diasem = dataframes[3]
df_mes = dataframes[4]
df_rhora = dataframes[6]
df_genero = dataframes[7]
df_lugar = dataframes[8]
df_medio = dataframes[9]
df_motivo = dataframes[10]
df_grupoedad = dataframes[11]

In [47]:
dataframes_list = [
    df_region, df_provincia, df_diasem, df_mes, df_rhora,
    df_genero, df_lugar, df_medio, df_motivo, df_grupoedad]

for i, df in enumerate(dataframes_list):
    # me quedo con las filas a partir de la fila 3 porque así me quedo solo con el contenido
    df = df.iloc[2:]
    
    # coloco el primer valor en su lugar, ya que viene en la columna equivocada
    df.iloc[0, 1] = df.iloc[0, 2]
    df.iloc[0,0] = 1
    
    # me quedo solo la primera columna
    df = df[[0,1]]
    df = df[:-1]
    df.columns = ['Valor', 'Significado']
    df['Valor'] = df['Valor'].astype('int64')

    # reinicio el índice
    df.reset_index(drop=True, inplace=True)
    
    # guardo el df transformado de nuevo en la lista
    dataframes_list[i] = df

# Asignar los DataFrames de vuelta a sus variables
df_region, df_provincia, df_diasem, df_mes, df_rhora, \
df_genero, df_lugar, df_medio, df_motivo, \
df_grupoedad = dataframes_list

In [50]:
## guardo mis data frames

dataframes = [
    (df_diasem, 'df_diasem.csv'),
    (df_mes, 'df_mes.csv'),
    (df_rhora, 'df_rhora.csv'),
    (df_region, 'df_region.csv'),
    (df_provincia, 'df_provincia.csv'),
    (df_lugar, 'df_lugar.csv'),
    (df_medio, 'df_medio.csv'),
    (df_genero, 'df_genero.csv'),
    (df_motivo, 'df_motivo.csv'),
    (df_grupoedad, 'df_grupoedad.csv')
]

for df, filename in dataframes:
    df.to_csv(f'../Datos/{filename}')